In [1]:
import numpy as np
from matplotlib.image import NonUniformImage
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sys
import os
import ROOT
from ROOT import gSystem, gInterpreter
import approximator.particles_trajectory_generator as ptg
import approximator.runner as ap_runner
import data.bunch_configuration as buc
path_to_old_library = "/home/rafalmucha/Pobrane/optic/opticsGeneratorOld/lib"
if "LD_LIBRARY_PATH" not in os.environ:
    os.environ["LD_LIBRARY_PATH"] = os.pathsep + path_to_old_library
else:
    os.environ["LD_LIBRARY_PATH"] += os.pathsep + path_to_old_library

sns.set_style("whitegrid")

home = "/home/rafalmucha/Pobrane/optic/"
file_name = "parametrization_6500GeV_0p4_185_reco"
approximator_name = "ip5_to_station_150_h_2_lhcb1"
path_to_project = home+"/optics_generator_python"
path_to_file = home+"/2016/" + file_name + ".root"

Welcome to JupyROOT 6.16/00


In [2]:
approximator = ap_runner.get_approximator(path_to_project, path_to_file, approximator_name)

AttributeError: apr0

Error in <TUnixSystem::FindDynamicLibrary>: LHCOpticsApproximator[.so | .dll | .dylib | .sl | .dl | .a] does not exist in .:/usr/lib64/root:
Warning in <TClass::Init>: no dictionary for class LHCOpticsApproximator is available
Warning in <TClass::Init>: no dictionary for class TMultiDimFet is available
Warning in <TClass::Init>: no dictionary for class LHCApertureApproximator is available
input_line_50:2:16: error: template argument for template type parameter must be a type
 std::auto_ptr<LHCOpticsApproximator> apr0 = std::auto_ptr<LHCOpticsAppr...
               ^~~~~~~~~~~~~~~~~~~~~
/usr/lib/gcc/x86_64-redhat-linux/8/../../../../include/c++/8/backward/auto_ptr.h:88:21: note: template parameter is declared here
  template<typename _Tp>
                    ^
input_line_50:2:60: error: use of undeclared identifier 'LHCOpticsApproximator'
 std::auto_ptr<LHCOpticsApproximator> apr0 = std::auto_ptr<LHCOpticsAppr...
                                                           ^


In [ ]:
# Parameters of bunch
x_mean = 0.0
sigma_x = 10e-6
theta_x_mean = 0
sigma_theta_x = 10e-6
y_mean = 0.0
sigma_y = 10e-6
theta_y_mean = 0.00
sigma_theta_y = 10e-6
pt_mean = -0.01
sigma_pt = 0.001

particles_number = 1000

In [ ]:
bunch_configuration = buc.BunchConfiguration.from_sigma(x_mean, sigma_x, particles_number,
                                                        theta_x_mean, sigma_theta_x, 1,
                                                        y_mean, sigma_y, 1,
                                                        theta_y_mean, sigma_theta_y, 1,
                                                        pt_mean, sigma_pt, 1)

In [ ]:
output = ptg.generate_random_particles(approximator, bunch_configuration)

In [ ]:
output["start"]

In [ ]:
# Names of segments, like MQXA.3R5
start_segment_name = "start"
end_segment_name = "end"

if start_segment_name != "start":
    output[start_segment_name].T[1] -= 1

In [ ]:
output_matrix = output[end_segment_name]
input_matrix = output[start_segment_name]

In [ ]:
index_map = {
    "x": 0,
    "theta x": 1,
    "y": 2,
    "theta y": 3,
    "t": 4,
    "pt": 5,
    "alfa x": 6,
    "alfa y": 7
}
meters = " [m]"
radians = " [rad]"
seconds = " [s]"
energy = " [TeV]"
no_unit = " "
unit_map = {
    "x": meters,
    "theta x": radians,
    "y": meters,
    "theta y": radians,
    "t": seconds,
    "pt": no_unit,
    "s": meters,
    "e": energy,
    "alfa x": radians,
    "alfa y": radians
}


def plot_histogram(x_name, y_name, x_name_prefix, y_name_prefix, x_matrix, y_matrix, plot_axes, plot_x_pos, plot_y_pos):
    
    title = "Histogram " + x_name_prefix + x_name + " and " + y_name_prefix + y_name + " parameter"
    
    # Get vectors and their properties
    
    x_index = index_map[x_name]
    y_index = index_map[y_name]
    
    vector_x = pd.Series(x_matrix.T[x_index], name=x_name_prefix + x_name)
    vector_y = pd.Series(y_matrix.T[y_index], name=y_name_prefix + y_name)
    
    print(np.min(vector_x), np.max(vector_x))
    
    bins_number = 50
    
    sns.distplot(vector_x, bins=bins_number, ax=axes[plot_x_pos][plot_y_pos], color='y').set_title(title)
    sns.distplot(vector_y, bins=bins_number, ax=axes[plot_x_pos][plot_y_pos], color='b').set_title(title)

In [ ]:
f, axes = plt.subplots(2, 2)

fig = plt.gcf()
fig.set_size_inches(20,20)

plot_histogram("x", "x", "In ", "Out ", input_matrix, output_matrix, axes, 0, 0)

plot_histogram("y", "y", "In ", "Out ", input_matrix, output_matrix, axes, 0, 1)

plot_histogram("theta x", "theta x", "In ", "Out ", input_matrix, output_matrix, axes, 1, 0)

plot_histogram("theta y", "theta y", "In ", "Out ", input_matrix, output_matrix, axes, 1, 1)
